In [1]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Database connection setup
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

db_url = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}"
engine = create_engine(db_url)

# Fetch symbols from the Asset table in PostgreSQL
symbols = []
with engine.connect() as conn:
    result = conn.execute(text("SELECT AssetCode FROM Asset where AssetStatus = 'Active'"))
    for row in result:
        symbols.append(row[0])

# Fetch and process data for each symbol
all_data = []

for symbol in symbols:
    try:
        # Download stock data for each symbol
        df = yf.download(tickers=symbol, period="5d", interval="1d")
        
        # If no data is returned, skip this symbol
        if df.empty:
            print(f"No data for symbol: {symbol}")
            continue

        # Keep only relevant columns, rename, and add Symbol column
        df = df[['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']].copy()
        df.columns = ['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Volume']
        df['Symbol'] = symbol
        df.reset_index(inplace=True)  # Flatten the Date index

        all_data.append(df)

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Combine all data into a single DataFrame with uniform columns
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Display a sample to verify data structure
    print("Sample data from combined DataFrame:")
    print(combined_df.head())

    # Define the table structure in PostgreSQL (create if not exists)
    table_name = 'pricestaging'
    combined_df.to_sql(table_name, con=engine, if_exists='replace', index=False)

    # Insert or update data in the staging table
    combined_df.to_sql(name=table_name, con=engine, if_exists='append', index=False, method='multi')

    print("Data fetch and insertion complete.")
else:
    print("No data fetched. Exiting.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

No data for symbol: USDCASH


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['GBPCASH']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")')
[*********************100%***********************]  1 of 1 completed


No data for symbol: GBPCASH


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

No data for symbol: CRSRDIDI


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TOAST']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


No data for symbol: TOAST


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['DIDI']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")')
[*********************100%***********************]  1 of 1 completed


No data for symbol: DIDI
Sample data from combined DataFrame:
                       Date  Adj_Close      Close       High        Low  \
0 2024-11-06 00:00:00+00:00  38.669998  38.669998  39.029999  36.930000   
1 2024-11-07 00:00:00+00:00  41.230000  41.230000  41.500000  40.070000   
2 2024-11-08 00:00:00+00:00  38.349998  38.349998  40.200001  38.139999   
3 2024-11-11 00:00:00+00:00  39.020000  39.020000  39.290001  38.615002   
4 2024-11-12 00:00:00+00:00  36.099998  36.099998  36.910000  36.009998   

        Open    Volume Symbol  
0  38.064999  15031800     JD  
1  40.145000  16561700     JD  
2  39.790001  13967600     JD  
3  39.080002  12044300     JD  
4  36.910000  20006243     JD  
Data fetch and insertion complete.
